# Reading data

In [ ]:
import pickle

# Define the file path
file_path_train = "measurements_by_patient_train.pkl"
file_path_test = "measurements_by_patient_test.pkl"

with open(file_path_train, 'rb') as file:
    train = pickle.load(file)

with open(file_path_test, 'rb') as file:
    test = pickle.load(file)

# Generating windows

This section create the windows for the new two horizons (90 and 120 minutes)

In [ ]:
import numpy as np

def get_windows_one_step_walk_forward(data, lookback_samples, pred_samples):
  """

    :param data:
    :param lookback_samples: Number of samples used to predict
    :param pred_samples: Prediction window (normally 30 or 60 minutes)
    :param backup: Save the progress each 100 patients
    :return: Get windows without missing values from one step sliding windows
    """
  x_per_patient = []
  y_per_patient = []

  for data_patient in data:

    # Creating the one step sliding window
    x = np.lib.stride_tricks.sliding_window_view(data_patient[:-pred_samples], lookback_samples)
    y = np.lib.stride_tricks.sliding_window_view(data_patient[lookback_samples:], pred_samples)

    # Removing rows with missing values
    nan_rows_x = np.isnan(x).any(axis=1)
    nan_rows_y = np.isnan(y).any(axis=1)
    x = x[~(nan_rows_x | nan_rows_y)]
    y = y[~(nan_rows_x | nan_rows_y)]

    x_per_patient.append(x)
    y_per_patient.append(y)

  x_result = np.concatenate(x_per_patient)
  y_result = np.concatenate(y_per_patient)

  return x_result, y_result



history_length = 8
horizons = [6, 8] #90 and 120 min

for hor in horizons:
    x_train, y_train = get_windows_one_step_walk_forward(train.values(), history_length, hor)
    x_test, y_test = get_windows_one_step_walk_forward(test.values(), history_length, hor)
    # NOTE: Replace with your filepath
    np.save('windows/x_train_windows_horizon_{0}.npy'.format(hor), x_train)     
    np.save('windows/y_train_windows_horizon_{0}.npy'.format(hor), y_train)
    np.save('windows/x_test_windows_horizon_{0}.npy'.format(hor), x_test)
    np.save('windows/y_test_windows_horizon_{0}.npy'.format(hor), y_test)
    
    

# Auxiliar Functions

## Windows

In [ ]:
import numpy as np 

def read_windows(horizon):
  file_x_train = f"windows/x_train_windows_horizon_{horizon}.npy"
  file_y_train = f"windows/y_train_windows_horizon_{horizon}.npy"
  file_x_test  = f"windows/x_test_windows_horizon_{horizon}.npy"
  file_y_test  = f"windows/y_test_windows_horizon_{horizon}.npy"
  
  print(f"Reading training files {file_x_train} and {file_y_train}...")
  print(f"Reading test files {file_x_test} and {file_y_test}...")
  
  x_train = np.load(file_x_train)
  y_train = np.load(file_y_train)
  x_test = np.load(file_x_test)
  y_test = np.load(file_y_test)

  # Using only the horizon measurement as a label
  y_train = y_train[:, -1]
  y_train = y_train.reshape((y_train.shape[0], 1))
  y_test = y_test[:, -1]
  y_test = y_test.reshape((y_test.shape[0], 1))

  return x_train, y_train, x_test, y_test



## LSTM and linear model architectures

In [ ]:
from keras.models import Model
from keras.layers import Dense, LSTM, GRU, Lambda, dot, concatenate, Activation, Input

# LSTM
class LSTMModel:
    def __init__(self, input_shape, nb_output_units, nb_hidden_units=128):
        self.input_shape = input_shape
        self.nb_output_units = nb_output_units
        self.nb_hidden_units = nb_hidden_units
        
    def __repr__(self):
        return 'LSTM_{0}_units_{1}_layers_dropout={2}_{3}'.format(self.nb_hidden_units, self.nb_layers, self.dropout, self.recurrent_dropout)
    
    def build(self):
        # input
        i = Input(shape=self.input_shape)

        # add LSTM layer
        x = LSTM(self.nb_hidden_units)(i)

        x = Dense(self.nb_output_units, activation=None)(x)

        return Model(inputs=[i], outputs=[x])
    
# Linear
class LinearModel:
    def __init__(self, input_shape, nb_output_units):
        self.input_shape = input_shape
        self.nb_output_units = nb_output_units

    def __repr__(self):
        return 'Linear'

    def build(self):
        i = Input(shape=self.input_shape)
        x = Dense(self.nb_output_units, activation=None)(i)

        return Model(inputs=[i], outputs=[x])


## Model Functions

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend as K
from tensorflow import keras
from keras.metrics import RootMeanSquaredError

#def RMSE(output, target):
#    output = K.cast(output, 'float32')
#    target = K.cast(target, 'float32')
#
#    return K.sqrt(K.mean((output - target) ** 2))

def build_model(model, weights=''):
  
  # build & compile model
  m = model.build()

  m.compile(loss=RootMeanSquaredError,
            optimizer=keras.optimizers.legacy.Adam(),
            metrics=[RootMeanSquaredError])
  if weights:
    print(f"Weights: {weights}")
    m.load_weights(weights)
  return m

def callbacks(filepath, early_stopping_patience):
  callbacks = []
  callbacks.append(ModelCheckpoint(filepath=filepath,
                                  monitor='RMSE',
                                  save_best_only=True,
                                  save_weights_only=True))
  callbacks.append(EarlyStopping(monitor='RMSE', patience=early_stopping_patience))
  return callbacks

In [ ]:

import numpy as np

def prepare_model_LSTM(history_length, nb_hidden_units=128, weights=''):
  model = LSTMModel(input_shape=(history_length, 1), nb_output_units=1, nb_hidden_units=nb_hidden_units)
  return build_model(model, weights)

def prepare_model_linear(history_length, weights=''):
  model = LinearModel(input_shape=(history_length,), nb_output_units=1)
  return build_model(model, weights)
 
 
 
def train(x_train, y_train, model, horizon, save_filepath = "", early_stopping_patience=50):

  history_length = 8
  x_train = np.reshape(x_train, (x_train.shape[0], history_length, 1))

  hist = model.fit(x_train, y_train,
                batch_size=32,
                epochs=500,
                #shuffle=True,
                callbacks=callbacks(save_filepath + str(horizon), 
                                    early_stopping_patience)
                )


  return hist

# Training

In this sections the models are trained for the new horizons

In [ ]:
history_length = 8 # 2h

x_train_6, y_train_6, x_test_6, y_test_6 = read_windows(horizon=6)
x_train_8, y_train_8, x_test_6, y_test_6 = read_windows(horizon=8)

LSTM_model = prepare_model_LSTM(history_length)
linear_model = prepare_model_linear(history_length)

# NOTE: Be sure that the save_filepath is a real path. If not, the callback
#won't save your model and you will lose your progress

print("########################################################################")
hist = train(x_train_6, y_train_6, LSTM_model, horizon=6, save_filepath="LSTM")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_8, y_train_8, LSTM_model, horizon=8, save_filepath="LSTM")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_6, y_train_6, linear_model, horizon=6, save_filepath="Linear")
print("########################################################################")
print()

print("########################################################################")
hist = train(x_train_8, y_train_8, linear_model, horizon=8, save_filepath="Linear")
print("########################################################################")
print()



# Results   

## Functions

In [ ]:
import matplotlib.pyplot as plt
def clarke_error_grid(ref_values, pred_values, title_string, show_plot=True):
    """
      This function takes in the reference values and the prediction values as lists and returns a list with each index corresponding to the total number
     of points within that zone (0=A, 1=B, 2=C, 3=D, 4=E) and the plot
    """
    #Checking to see if the lengths of the reference and prediction arrays are the same
    assert (len(ref_values) == len(pred_values)), "Unequal number of values (reference : {0}) (prediction : {1}).".format(len(ref_values), len(pred_values))

    #Checks to see if the values are within the normal physiological range, otherwise it gives a warning
    if max(ref_values) > 500 or max(pred_values) > 500:
        print("Input Warning: the maximum reference value {0} or the maximum prediction value {1} exceeds the normal physiological range of glucose (<400 mg/dl).".format(max(ref_values), max(pred_values)))
    if min(ref_values) < 0 or min(pred_values) < 0:
        print( "Input Warning: the minimum reference value {0} or the minimum prediction value {1} is less than 0 mg/dl.".format(min(ref_values),  min(pred_values)))

    values_out_grid = sum(value > 500 for value in pred_values) + sum(value < 0 for value in pred_values)
    print(f"Number of values outside the grid: {values_out_grid}")

    if show_plot:
      #Clear plot
      plt.clf()

      #Set up plot
      plt.scatter(ref_values, pred_values, marker='o', color='black', s=8)
      plt.title(title_string + " Clarke Error Grid")
      plt.xlabel("Reference Concentration (mg/dl)")
      plt.ylabel("Prediction Concentration (mg/dl)")
      plt.xticks([0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])
      plt.yticks([0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500])
      plt.gca().set_facecolor('white')

      #Set axes lengths
      plt.gca().set_xlim([0, 500])
      plt.gca().set_ylim([0, 500])
      plt.gca().set_aspect((500)/(500))

      #Plot zone lines
      plt.plot([0,500], [0,500], ':', c='black')                      #Theoretical 45 regression line
      plt.plot([0, 175/3], [70, 70], '-', c='black')
      #plt.plot([175/3, 320], [70, 500], '-', c='black')
      plt.plot([175/3, 500/1.2], [70, 500], '-', c='black')           #Replace 320 with 400/1.2 because 100*(400 - 400/1.2)/(400/1.2) =  20% error
      plt.plot([70, 70], [84, 500],'-', c='black')
      plt.plot([0, 70], [180, 180], '-', c='black')
      plt.plot([70, 290],[180, 500],'-', c='black')
      # plt.plot([70, 70], [0, 175/3], '-', c='black')
      plt.plot([70, 70], [0, 56], '-', c='black')                     #Replace 175.3 with 56 because 100*abs(56-70)/70) = 20% error
      # plt.plot([70, 500],[175/3, 320],'-', c='black')
      plt.plot([70, 500], [56, 320],'-', c='black')
      plt.plot([180, 180], [0, 70], '-', c='black')
      plt.plot([180, 500], [70, 70], '-', c='black')
      plt.plot([240, 240], [70, 180],'-', c='black')
      plt.plot([240, 500], [180, 180], '-', c='black')
      plt.plot([130, 180], [0, 70], '-', c='black')

      #Add zone titles
      plt.text(30, 15, "A", fontsize=15)
      plt.text(370, 260, "B", fontsize=15)
      plt.text(280, 370, "B", fontsize=15)
      plt.text(160, 370, "C", fontsize=15)
      plt.text(160, 15, "C", fontsize=15)
      plt.text(30, 140, "D", fontsize=15)
      plt.text(370, 120, "D", fontsize=15)
      plt.text(30, 370, "E", fontsize=15)
      plt.text(370, 15, "E", fontsize=15)

    #Statistics from the data
    zone = [0] * 5
    for i in range(len(ref_values)):
        if (ref_values[i] <= 70 and pred_values[i] <= 70) or (pred_values[i] <= 1.2*ref_values[i] and pred_values[i] >= 0.8*ref_values[i]):
            zone[0] += 1    #Zone A

        elif (ref_values[i] >= 180 and pred_values[i] <= 70) or (ref_values[i] <= 70 and pred_values[i] >= 180):
            zone[4] += 1    #Zone E

        elif ((ref_values[i] >= 70 and ref_values[i] <= 290) and pred_values[i] >= ref_values[i] + 110) or ((ref_values[i] >= 130 and ref_values[i] <= 180) and (pred_values[i] <= (7/5)*ref_values[i] - 182)):
            zone[2] += 1    #Zone C
        elif (ref_values[i] >= 240 and (pred_values[i] >= 70 and pred_values[i] <= 180)) or (ref_values[i] <= 175/3 and pred_values[i] <= 180 and pred_values[i] >= 70) or ((ref_values[i] >= 175/3 and ref_values[i] <= 70) and pred_values[i] >= (6/5)*ref_values[i]):
            zone[3] += 1    #Zone D
        else:
            zone[1] += 1    #Zone B

    return zone

In [ ]:
def get_values_per_zone(values):
  keys = ['A', 'B', 'C', 'D', 'E']
  return {key: value for key, value in zip(keys, values)}

def show_results(model, x_test, y_test):

  
  test_score = model.evaluate(x_test, y_test)
  print(f"RMSE: {test_score[1]}")
  print()

  #PREDICTION
  print("########################################################################")
  print("Predicting BG values...")
  BG_predicted_values = model.predict(x_test)
  print("########################################################################")
  print()

  # Clarke error grid
  values = clarke_error_grid(y_test, BG_predicted_values, f"Clarke Error Grid", show_plot=True)


  values_zones = get_values_per_zone(values)
  predicted_values_len = y_test.shape[0]
  perc_values_zones = [value / predicted_values_len * 100 for value in values_zones.values()]

  perc_values_zones = get_values_per_zone(perc_values_zones)

  return test_score, perc_values_zones

## Example

In [ ]:
LSTM_model = prepare_model_LSTM(history_length, weights="LSTM6")    #En weights irían los pesos calculados en el train
x_train_6, y_train_6, x_test_6, y_test_6 = read_windows(horizon=6)
show_results(LSTM_model, x_test_6, y_test_6)